# Get started
------
https://www.runoob.com/python3/python3-iterator-generator.html

### 生成器
------
在 Python 中，使用了 yield 的函数被称为生成器（generator）。

跟普通函数不同的是，生成器是一个返回迭代器的函数，只能用于迭代操作，更简单点理解生成器就是一个迭代器。

在调用生成器运行的过程中，每次遇到 yield 时函数会暂停并保存当前所有的运行信息，返回 yield 的值, 并在下一次执行 next() 方法时从当前位置继续运行。

调用一个生成器函数，返回的是一个迭代器对象。

__以下实例使用 yield 实现斐波那契数列:__

In [10]:
import sys

def fibonacci(n): # 生成器函数 - 斐波那契
    a, b, counter = 0, 1, 0
    while True:
        if (counter > n):
            return
        yield a
        a, b = b, a + b
        counter += 1

f = fibonacci(10) # f 是一个迭代器，由生成器返回生成（生成器是一类特殊的迭代器）
while True:
    try:
        print(next(f), end=' ')
    except StopIteration:
        sys.exit()

0 1 1 2 3 5 8 13 21 34 55 

SystemExit: 

# 闭合器和生成器 Closures & Generators
-----
建议结合书目看/回顾

### Get started，利用re做复数转换

In [6]:
import re

In [9]:
# 定义几种(4种)基于规则的复数转换方式
def plural(noun): # plural 复数形式、复数的
    if re.search('[sxz]', noun): # 回忆上一章节，re匹配不到将会返回None
        return re.sub('$', 'es', noun) # 基于re的字符串替换 re.sub()
    
    # []中的^有特别含义：非;这里指的是h前紧跟的字母不是这些里面的一个。
    elif re.search('[^aeioudgkprt]h$', noun): # 发音以非aeiou等+h结尾，要加es
        return re.sub('$', 'es', noun)
    # 如果某个单词以发 I 音的字母 Y 结尾，将 Y 改成 IES;如果 Y 与某个原因字母组合发其它音的话，只需加上 S 。因此 vacancy 变成 vacancies，但 day 变成 days 。
    elif re.search('[^aeiou]y$', noun):
        return re.sub('y$', 'ies', noun)
    else:
        return noun + 's'

In [5]:
plural('peach')

'peaches'

以上四种简单的规则并为实现sheep、man、deer等的复数

看一下正则表达式替换的细节

In [26]:
re.search('[abc]', 'Mark')

<re.Match object; span=(1, 2), match='a'>

In [27]:
re.sub('[abc]', 'o', 'Mark')

'Mork'

In [31]:
# re.sub 会 替换 所有的 匹配项, 而不仅仅是第一个匹配项
re.sub('[abc]', 'o', 'caps')

'oops'

看看“否定”正则表达式的更多细节

In [35]:
re.search('[^aeiou]y$', 'vacancy')

<re.Match object; span=(5, 7), match='cy'>

In [39]:
re.search('[^aeiou]y$', 'boy')

In [42]:
re.sub('y$', 'ies', 'vacancy')

'vacancies'

In [43]:
re.sub('y$', 'ies', 'agency')

'agencies'

In [46]:
# 可以将该两条正则表达式合并起来(一条 查找是否应用该规则，
# 另一条实际应用规则)，使其成为一条 正则表达式
re.sub('([^aeiou])y$', r'\1ies', 'vacancy')
# 第一个分组用于保存字母y之前的字符
# 第二个分组使用了新的语法，表示将之前记住的分组放到这里

'vacancies'

### 函数列表 A List Of Functions

现在要增加一些抽象层次的内容。我们开始时定义了一系列规则:如果这样，那样做;否则前往下一条规则。

现在让我们对部分程序进行临时的复杂化，以简化另一部分。

模块化 附加抽象层

Python中一切都是对象，包括函数，故而数据结构rules包含了函数——不是函数名称，而是实际的函数对象。

In [153]:
import re 

In [154]:
def match_sxz(noun):
    return re.search('[sxz]$', noun)

In [155]:
def apply_sxz(noun):
    return re.sub('$', 'es', noun)

In [156]:
def match_h(noun):
    return re.search('[^aeioudgkprt]h$', noun)

In [157]:
def apply_h(noun):
    return re.sub('$', 'es', noun)

In [158]:
def match_y(noun):
    return re.search('[^aeiou]y$', noun)

In [159]:
def apply_y(noun):
    return re.sub('y$', 'ies', noun)

In [160]:
def match_default(noun):
    return True

In [161]:
def apply_default(noun):
    return noun + 's'

In [162]:
# 函数列表

rules = ((match_sxz, apply_sxz),
        (match_h, apply_h),
        (match_y, apply_y),
        (match_default, apply_default)
        )

In [163]:
type(rules)

tuple

In [166]:
def plural(noun):
    for match_rule, apply_rule in rules:
        if match_rule(noun):
            return apply_rule(noun)

In [167]:
# 测试
plural('word')

'words'

### 匹配模式列表 (A List Of Patterns)  （）
------
1. 如果要新加规则，引入其

2. 闭合 Closures

【其实并不是真的有必要为每个匹配和应用规则定义各自的命名函数。】

它们从未直接被调用，而只是被添加到 rules 序列并从该处被调用。

此外，每个函数遵循任意模式的其中之一。所有的匹配函数调用re.search()，而所有的应用函数调用 re.sub()。

让我们将模式排除在考虑因素之外，使新规则定义更加简单。

In [168]:
import re

In [169]:
# 用于动态创建其他函数，接受p,s,r三个参数
def build_match_and_apply_functions(pattern, search, replace):
    def matches_rule(word):# 1
        return re.search(pattern, word)
    def apply_rule(word): # 2
        return re.sub(search, replace, word)
    return (matches_rule, apply_rule) # 3

1. build_match_and_apply_functions() 函数用于动态创建函数。它接受 pattern、 search 和 replace 三个参数，并定义了 matches_rule() 函数，该函数通过传给 build_match_and_apply_functions() 函数的 pattern 及【传递给 所创建的matchs_rules()函数的word】调用 re.search() 函数。


2. apply_rule函数的创建工作采用了同样的方式。应用函数只接收一个参数。【在动态函数中使用外部参数值的技术称为闭合【closures】】。【基本上，常量的创建工作都在创建应用函数过程中完成:它接受一个参数 (word)，但实际操作还加上了另外两个值(search 和 replace)，该两个值都在定义应用函数时进行设置。】


3. 最后，build_match_and_apply_functions() 函数返回一个包含两个值的元组:即刚才所创建的两个函数。在这些函数中定义的常量( match_rule() 函数中的 pattern 函数，apply_rule()函数中的 search 和 replace )与这些函数呆在一起，即便是在 从 build_match_and_apply_functions() 中返回后也一样。这真 是非常酷的一件事情。


但如果此方式导致了难以置信的混乱(应该是这样，它确实有点奇怪)，在看看如何使用之后可能会清晰一些。


In [212]:
# Go on...

patterns = \
(
    ('[sxz]$',           '$',  'es'), # 1
    ('[^aeioudgkprt]h$', '$',  'es'),
    ('(qu|[^aeiou])y$',  'y$', 'ies'),
    ('$',                '$',  's' )
# 2
)
# 3
rules = [build_match_and_apply_functions(pattern, search, replace) \
        for (pattern, search, replace) in patterns]

1. 复数构成的“规则——rules”现在被定义为一个字符串的元组的元组 (而非函数)。每个元组的第一个字符串是在re.search()中用于判断该规则是否匹配的正则表达式。

各组中的第二和第三个字符串是在 re.sub() 中将实际用于使用规则将名词转换为复数形式的搜索和替换表达式。
2. 此处的后备规则略有变化。在前例中，match_default() 函数 仅返回 True，意思是如果更多的指定规则无一匹配，代码将简 单地向给定词汇的尾部添加一个 s。本例则进行了一些功能等 同的操作。最后的正则表达式询问单词是否有一个结尾(\$ 匹 配字符串的结尾)。当然，每个字符串都有一个结尾，甚至是 空字符串也有，因此该规则将始终被匹配。因此，【它实现了 match_default() 函数同样的目的，始终返回 True:它确保了如果没有更多的指定规则用于匹配，代码将向给定单词的尾部增加一个s。】
3. （可略看）本行代码非常神奇。它以 patterns 中的字符串序列为参数， 并将其转换为一个函数序列(列表推导式)。怎么做到的?通过将字符串“映射”到 build_match_and_apply_functions() 函数。也就是说， 它接受每组三重字符串为参数，并将该三个字符串作为实参调用build_match_and_apply_functions() 函数。 【build_match_and_apply_functions() 函数返回一个包含两个函数的元组。】也就是说该规则最后的结尾与前例在功能上是等价的:一个元组列表，每个元组都是一对函数。第一个函数是调用 re.search() 的匹配函数;而第二个函数调用 re.sub() 的应 用函数。


In [213]:
rules[0]

(<function __main__.build_match_and_apply_functions.<locals>.matches_rule(word)>,
 <function __main__.build_match_and_apply_functions.<locals>.apply_rule(word)>)

In [214]:
# 此版本脚本的最前面是主入口点——plural()函数

def plural(noun):
    for matches_rule, apply_rule in rules:
        if matches_rule(noun):
            return apply_rule(noun)

In [215]:
plural('watch')

'watches'

In [42]:
# 总结：为了演示Python特性，有些过于复杂

### A File Of Patterns 模式文件 
------
目前，已经排除了重复代码，增加了足够的抽象性，因此复数
形式规则可以字符串列表的形式进行定义。下一个逻辑步骤是
将这些字符串放入一个单独的文件中，因此可独立于使用它们
的代码来进行维护。Good...

文件：plural.ipynb   plural-rules.txt

In [226]:
import re

def build_match_and_apply_functions(patter, search, replace):
    def matches_rule(word):
        return re.search(pattern, word)
    def apply_rule(word):
        return re.sub(search, replace, word)
    return (matches_rule, apply_rule)

# 从外部rules独立文件中读取内容
rules = []
with open('plural-rules.txt', encoding='utf-8') as pattern_file:
    for line in pattern_file:
        pattern, search, replace = line.split(None, 3) # 4
        rules.append(build_match_and_apply_functions(pattern,search,replace))

In [232]:
# 看一下rules规则列表
len(rules) # 4 个 规则

4

In [228]:
# 函数统一入口

def plural(noun):
    for matches_rule, apply_rule in rules:
        if matches_rule(noun):
            return apply_rule(noun)
        
plural('wch')

'wches'

最后，将 pattern 、 search 和 replace 传入build_match_and_apply_functions() 函数，它将返回一个函数 的元组。将该元组添加到 rules 列表，最终 rules 将储存 plural() 函数所预期的匹配和应用函数列表。 此处的改进是将复数形式规则独立地放到了一份外部文件中， 因此可独立于使用它的代码单独对规则进行维护。代码是代 码，数据是数据，生活更美好

### Generators 生成器（存档点）
--------
如果有个通用 plural()函数解析规则文件不就更棒了吗?获取规则，检查匹配，应用相应的转换，进入下一条规则。这是 plural() 函数所必须完成的事，也是plural() 函数必须做的事。

In [193]:
def rules(rules_filename):
    with open(rules_filename, encoding='utf-8') as pattern_file:
        for line in pattern_file:
            pattern,search,replace = line.split(None,3) # 0或多个空白分隔符，获取3列
            yield build_match_and_apply_functions(pattern,search,replace)

def plural(noun, rules_filename='plural-rules.txt'):
    for m_r, a_r in rules(rules_filename):
        if m_r(noun):
            return a_r(noun)
    raise ValueError('no matching rule for {0}'.format(noun))

这段代码到底如何运作？我们先看一个交互式（更简单）的例子。

In [124]:
def make_counter(x):
    print('函数开始:')
    while True:
        yield x
        print('x正在增加...')
        x += 1
        
# make_counter中出现的yield命令的意思是这不是一个普通的函数。
# 它是一次生成一个值的特殊类型函数。可以将其看作为可恢复函数。
# 调用该函数将返回一个可用于生成连续x值的生成器【Generator】

In [125]:
counter = make_counter(2) 
# 创建make_counter生成器的实例
# 并不实际执行函数代码。

counter
# 一个生成器对象。

<generator object make_counter at 0x10750bb88>

In [126]:
next(counter)
# next()函数以第一个生成器对象为参数，并返回其下一个值。
# 对counter生成器【第一次调用next()】，
# 它针对第一条yield语句执行make_counter()中的代码，然后返回所产生的值。
# 故该代码输出将为2，因为其仅通过调用make_counter(2)对生成器进行初始化创建。

函数开始:


2

In [127]:
next(counter)
# 对同一个Generator对象反复调用next()将确切地从上次调用的位置开始继续，
# 直到下一条yield语句。
# 所有的变量、局部数据等内容在yield时被保存，在next()时被回复。
# 下一行代码等待被执行以调用print以打印出incrementing x。
# 之后，执行语句x+=1。
# 然后继续通过while再次循环，而它再次遇到yield，将保存所有一切状态，
# 并返回当前x的值（当前为3）

x正在增加...


3

In [70]:
next(counter) # 同理

x正在增加...


4

In [107]:
def make_counter(x):
    print('倒计时...')
    while(x > 0):
        print(x)
        x = x - 1


In [109]:
make_counter(5)

倒计时...
5
4
3
2
1


In [135]:
# 拿来对比
def make_counter(x):
    print('函数开始:')
    while True:
        yield x
        print('x正在增加...')
        x += 1
        

In [140]:
def mc_generator_yield(x):
    print('倒计时...')
    while x >= 0:
        yield x # 要标记yield哪个变量
        x = x - 1

In [143]:
counter = mc_generator_yield
for n in counter(5):
    print(n)

倒计时...
5
4
3
2
1
0


In [139]:
next(counter)

4

### 斐波娜契生成器
-----
1. “yield暂停一个函数”。"next()"从其暂停处恢复其运行。


2. 将不用next()实现Generators

In [71]:
def fib(max):
    a,b=0,1
    while a<max:
        yield a # 暂停
        a , b = b, a + b

In [73]:
for n in fib(20): # 方式1:for循环中使用生成器
    print(n)

0
1
1
2
3
5
8
13


In [75]:
print(type(fib(100)))
list(fib(100)) # 方式2:list中使用生成器

<class 'generator'>


[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89]

### 复数规则生成器 A Plural Rule Generator
1. 首先去回顾下加入yield的plural()函数

In [240]:
re.sub?

In [247]:
def build_match_and_apply_functions(pattern, search, replace):
    def match_rules(word):
        return re.search(pattern, word)
    def apply_rules(word):
        return re.sub(search, replace, word)
    return (match_rules, apply_rules)

# rules是按照需求连续征程匹配和应用函数的生成器
def rules(rules_filename):
    with open(rules_filename, encoding='utf-8') as pattern_file:
        for line in pattern_file:
            pattern,search,replace = line.split(None,3) # 1：获取三‘列’的值
            yield build_match_and_apply_functions(pattern,search,replace) # 2：该函数动态创建两个函数

def plural(noun, rules_filename='plural-rules.txt'):
    for m_r, a_r in rules(rules_filename): # 3 ⭐️
        if m_r(noun):
            # print(m_r(noun))
            # print(a_r(noun))
            return a_r(noun)
    raise ValueError('no matching rule for {0}'.format(noun))
    
# 3 由于rules()是生成器，可直接在for循环中使用它。
#   对for循环的第一次遍历，可以调用rules()函数打开匹配模式文件，读取第一行——从该行的模式动态创建一个匹配
#   函数和应用函数，然后生成动态创建的函数。
#   对for循环的第二次遍历，将会精确回到rules()上次离开的位置——for line in pattern_file循环的中间。
#   要进行的第一项工作是读取文件（仍处于打开状态）的下一行，基于改行的模式动态创建另一匹配和应用函数，然后
#   生成两个函数m_r和a_r

1. 通过第四步获得了什么呢?启动时间。在第四步中引入 plural4 模块（本文为plural函数）时，它读取了整个模式文件，并创建了一份所有可能规则 的列表，甚至在考虑调用 plural() 函数之前。有了生成器，可 以轻松地处理所有工作:可以读取规则，创建函数并试用它 们，如果该规则可用甚至可以不读取文件剩下的部分或创建更 多的函数。
2. 失去了什么?性能!每次调用 plural() 函数，rules() 生成器 将从头开始——这意味着重新打开模式文件，并从头开始读 取，每次一行。
要是能够两全其美多好啊:最低的启动成本(无需对 import 执 行任何代码)，同时 最佳的性能(无需一次次地创建同一函 数)。哦，还需将规则保存在单独的文件中(因为代码和数据 要泾渭分明)，还有就是永远不必两次读取同一行。
要实现该目标，必须建立自己的生成器。在进行此工作之前， 必须对 Python 的类进行学习。

In [248]:
plural('watch')

'watches'

### 总结
----

1. 生成器的构造：yield
2. 动态函数：函数内创建函数
3. 闭合（在动态函数中使用外部参数：本例是noun）的含义与使用